In [2]:
!pip install -U awswrangler pyqt5 pyqtwebengine bokeh ipython pylint
!pip install sagemaker-data-insights==0.4.0


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from time import sleep
import time
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker import hyperparameters
from sagemaker.tuner import ContinuousParameter, IntegerParameter, HyperparameterTuner
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import TuningStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker import Model
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.workflow.model_step import ModelStep

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

s3_client = boto3.client("s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Create a Pipeline Session
pipeline_session = PipelineSession()

In [5]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
mse_threshold = ParameterFloat(name="MseThreshold", default_value=6.0)

In [6]:
import pipe_tools

In [7]:
def train_filter(partitions):
    return (int(partitions["year"]) == 2012) 

def val_filter(partitions):
    return (int(partitions["year"]) == 2013) and (int(partitions["month"]) <= 2) 

def batch_filter(partitions):
    return (int(partitions["year"]) == 2012) and (int(partitions["month"]) <= 4) and (int(partitions["month"]) > 2)

In [8]:
pq_data = pipe_tools.get_pqdata(bucket, train_filter, val_filter, batch_filter)

/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)
/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)
/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)
/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


In [9]:
pq_data['val']

,kWh,weekday,hour,temperature,Holiday
timestamp,,,,,
2013-01-01 00:00:00,0.250960,1.0,0.0,7.01,5.0
2013-01-01 01:00:00,0.211900,1.0,1.0,7.49,5.0
2013-01-01 02:00:00,0.177964,1.0,2.0,7.16,5.0
2013-01-01 03:00:00,0.149907,1.0,3.0,7.04,5.0
2013-01-01 04:00:00,0.133761,1.0,4.0,6.29,5.0
...,...,...,...,...,...
2013-02-28 19:00:00,0.400435,3.0,19.0,4.17,-1.0
2013-02-28 20:00:00,0.394198,3.0,20.0,4.36,-1.0
2013-02-28 21:00:00,0.376216,3.0,21.0,3.90,-1.0


In [10]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

# record identifier and event time feature names
record_identifier_feature_name = "timestamp"
event_time_feature_name = "EventTime"

pq_data2 = pq_data.copy()

f_groups = pipe_tools.setup_feature_groups(pq_data2, feature_store_session, event_time_feature_name)

In [11]:
# You can modify the following to use a bucket of your choosing
feat_s3_bucket_name = feature_store_session.default_bucket()

In [12]:
%%time
prefix = "lgbm-fs-final"

dataset_uris = {}
for k in pq_data.keys():
    uri_prefix = f"s3://{feat_s3_bucket_name}/{prefix}/{k}_input/"
    dataset_uris[k] = uri_prefix
    pq_data[k].to_parquet(f"{k}_dataset.parquet")
    sess.upload_data(
        f"{k}_dataset.parquet", bucket=bucket, key_prefix=prefix + f"/{k}_input"
    )

dataset_uris

CPU times: user 38.3 ms, sys: 7.22 ms, total: 45.5 ms
Wall time: 344 ms


{'train': 's3://sagemaker-us-east-1-645257835735/lgbm-fs-final/train_input/',
 'val': 's3://sagemaker-us-east-1-645257835735/lgbm-fs-final/val_input/',
 'batch': 's3://sagemaker-us-east-1-645257835735/lgbm-fs-final/batch_input/'}

In [13]:
pq_data['val']

,kWh,weekday,hour,temperature,Holiday
timestamp,,,,,
2013-01-01 00:00:00,0.250960,1.0,0.0,7.01,5.0
2013-01-01 01:00:00,0.211900,1.0,1.0,7.49,5.0
2013-01-01 02:00:00,0.177964,1.0,2.0,7.16,5.0
2013-01-01 03:00:00,0.149907,1.0,3.0,7.04,5.0
2013-01-01 04:00:00,0.133761,1.0,4.0,6.29,5.0
...,...,...,...,...,...
2013-02-28 19:00:00,0.400435,3.0,19.0,4.17,-1.0
2013-02-28 20:00:00,0.394198,3.0,20.0,4.36,-1.0
2013-02-28 21:00:00,0.376216,3.0,21.0,3.90,-1.0


In [15]:
from sagemaker import get_execution_role
role = get_execution_role()

In [16]:
for f in f_groups:
    f.create(
        s3_uri=f"s3://{feat_s3_bucket_name}/{prefix}",
        record_identifier_name=record_identifier_feature_name,
        event_time_feature_name=event_time_feature_name,
        role_arn=role,
        enable_online_store=True,
    )

for f in f_groups:
    pipe_tools.wait_for_feature_group_creation_complete(feature_group=f)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup train-feature-group-02-07-40-41 successfully created.
FeatureGroup val-feature-group-02-07-40-41 successfully created.
FeatureGroup batch-feature-group-02-07-40-41 successfully created.


In [17]:
train_data = sagemaker.inputs.TrainingInput(
    dataset_uris['train'],
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)

validation_data = sagemaker.inputs.TrainingInput(
    dataset_uris['val'],
    distribution="FullyReplicated",
    content_type="application/x-parquet",
    s3_data_type="S3Prefix",
)

data_channels = {"train": train_data, 'validation': validation_data}

eval_data = ParameterString(
    name="InputDataUrl",
    default_value=dataset_uris['val'],
)

In [18]:
train_model_id = "lightgbm-regression-model"
train_model_version = "2.1.0"
train_scope = "training"
training_instance_type = "ml.m5.xlarge"

model_uri = pipe_tools.setup_uris(train_model_id, train_model_version, train_scope, training_instance_type)

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)
hyperparameters["num_boost_round"] = "500"  
print(hyperparameters)

{'num_boost_round': '500', 'early_stopping_rounds': '30', 'metric': 'auto', 'learning_rate': '0.009', 'num_leaves': '67', 'feature_fraction': '0.74', 'bagging_fraction': '0.53', 'bagging_freq': '5', 'max_depth': '11', 'min_data_in_leaf': '26', 'max_delta_step': '0.0', 'lambda_l1': '0.0', 'lambda_l2': '0.0', 'boosting': 'gbdt', 'min_gain_to_split': '0.0', 'tree_learner': 'serial', 'feature_fraction_bynode': '1.0', 'is_unbalance': 'False', 'max_bin': '255', 'tweedie_variance_power': '1.5', 'num_threads': '0', 'verbosity': '1', 'use_dask': 'False'}


In [19]:
use_amt = True

In [20]:
if train_model_id == "lightgbm-regression-model":
    hyperparameter_ranges = {
        "learning_rate": ContinuousParameter(1e-4, 1, scaling_type="Logarithmic"),
        "num_boost_round": IntegerParameter(2, 30),
        "early_stopping_rounds": IntegerParameter(2, 30),
        "num_leaves": IntegerParameter(10, 50),
        "feature_fraction": ContinuousParameter(0, 1),
        "bagging_fraction": ContinuousParameter(0, 1),
        "bagging_freq": IntegerParameter(1, 10),
        "max_depth": IntegerParameter(5, 30),
        "min_data_in_leaf": IntegerParameter(5, 50),
    }

In [21]:
from sagemaker.estimator import Estimator
training_output_path = f"s3://{feat_s3_bucket_name}/LGBMTrain"
LGBM_Estimator = Estimator(
    role=role,
    image_uri=model_uri['image'],
    source_dir=model_uri['source'],
    model_uri=model_uri['model'],
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=training_output_path,
    #sagemaker_session = pipeline_session
)

In [ ]:
from sagemaker.utils import name_from_base

training_job_name = name_from_base(f"SmartGrid-{train_model_id}-training")

if use_amt:
    tuner = HyperparameterTuner(
        LGBM_Estimator,
        "rmse",
        hyperparameter_ranges,
        [{"Name": "rmse", "Regex": "rmse: ([0-9\\.]+)"}],
        max_jobs=10,
        max_parallel_jobs=2,
        objective_type="Minimize",
        base_tuning_job_name=training_job_name,
    )

    hpo_args = tuner.fit(inputs=data_channels)
    
    step_tuning = TuningStep(
        name="HPTuning",
        step_args=hpo_args,
    )

else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    LGBM_Estimator.fit(
        inputs=data_channels, logs=True, job_name=training_job_name
    )

In [26]:
sagemaker_obj = boto3.client("sagemaker")
model_name = tuner.best_training_job()
print(model_name)


info = sagemaker_obj.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]


SmartGrid-lightgbm-r-240302-0742-002-13c5079b


In [ ]:
# Creating 2 SageMaker Models
model_prefix = training_job_name.replace("-tra", "")

In [ ]:
inference_instance_type = "ml.m5.large"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope="inference"
)

In [ ]:
best_model = Model(
    image_uri=deploy_image_uri,
    model_data=step_tuning.get_top_model_s3_uri(
        top_k=0, s3_bucket=feat_s3_bucket_name, prefix=model_prefix
    ),
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_first = ModelStep(
    name="CreateBestModel",
    step_args=best_model.create(instance_type=inference_instance_type),
)

second_best_model = Model(
    image_uri=deploy_image_uri,
    model_data=step_tuning.get_top_model_s3_uri(
        top_k=1, s3_bucket=feat_s3_bucket_name, prefix=model_prefix
    ),
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_second = ModelStep(
    name="CreateSecondBestModel",
    step_args=second_best_model.create(instance_type=inference_instance_type),
)

In [ ]:
%%writefile code/evaluate.py

"""Evaluation script for measuring mean squared error."""
import json
import logging
import pathlib
import tarfile
import joblib 

import numpy as np
import pandas as pd
import pyarrow as pa
import xgboost

from sklearn.metrics import mean_squared_error

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


if __name__ == "__main__":
    logger.debug("Starting evaluation.")
    model_path = "/opt/ml/processing/model/model.tar.gz"
    t = tarfile.open(model_path, 'r:gz')
    t.extractall()

    logger.debug("Loading lgbm model.")
    model = joblib.load(model_file_path)

    logger.debug("Reading test data.")
    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)

    logger.debug("Reading test data.")
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = xgboost.DMatrix(df.values)

    logger.info("Performing predictions against test data.")
    predictions = model.predict(X_test)

    logger.debug("Calculating mean squared error.")
    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mse": {"value": mse, "standard_deviation": std},
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    logger.info("Writing out evaluation report with mse: %f", mse)
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

In [ ]:
# A ProcessingStep is used to evaluate the performance of a selected model from the HPO step. In this case, the top performing model
# is evaluated. Based on the results of the evaluation, the model is registered into the Model Registry using a ConditionStep.

script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name=f"{base_job_prefix}/script-tuning-step-eval",
    sagemaker_session=pipeline_session,
    role=role,
)

evaluation_report = PropertyFile(
    name="BestTuningModelEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

processor_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(
                top_k=0, s3_bucket=feat_s3_bucket_name, prefix=model_prefix
            ),
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="code/evaluate.py",
)

# This can be extended to evaluate multiple models from the HPO step
step_eval = ProcessingStep(
    name="EvaluateTopModel",
    step_args=processor_args,
    property_files=[evaluation_report],
    cache_config=cache_config,
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

In [ ]:
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value",
    ),
    right=6.0,
)

In [ ]:

# Register the model in the Model Registry
# Multiple models can be registered into the Model Registry using multiple ModelSteps. These models can either be added to the
# same model package group as different versions within the group or the models can be added to different model package groups.

register_args = best_model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)


step_register_best = ModelStep(name="RegisterBesLGBTModel", step_args=register_args)

In [ ]:
pipeline = Pipeline(
    name="tuning-step-pipeline",
    parameters=[
        processing_instance_count,
        training_instance_type,
        input_data,
        model_approval_status,
    ],
    steps=[
        step_tuning,
        step_create_first,
        step_create_second,
        step_eval,
        step_cond,
    ],
    sagemaker_session=pipeline_session,
)

In [27]:
import json

definition = json.loads(pipeline.definition())
definition

NameError: name 'pipeline' is not defined

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
pipeline.start()